In [1]:
import numpy as np
from numpy import *
from random import shuffle
import math
import pandas as pd
from sklearn.metrics import r2_score

In [2]:
def data_generate(M,P_PDG,W,N,K,Noise):
    #W是原数据集,代表哪些代理之间有连接，即网络
    #N是W的行数，即代理的个数
    #代理人的博弈策略
    player = zeros((M+1,N))
    #原始博弈策略放在第一行，随机设置
    for i in range(N):
        if random.random()<=0.5:
            player[0,i] = 1 #1代表合作
        else:
            player[0,i] = 0 #0代表不合作
    #计算每个节点的收益
    #M轮博弈
    F = zeros((1,N))
    G = zeros((M,N))
    A = zeros((N,M,N))
    for t in range(M):
        for i in range(N):
            if player[t,i] == 1:
                s1 = array([[1],[0]])
            else:
                s1 = array([[0],[1]])
            for j in range(N):
                if player[t,j] == 0:
                    s2 = array([[0],[1]])
                else:
                    s2 = array([[1],[0]])
                F[0,j] = ((s1.T).dot(P_PDG)).dot(s2) #如果代理i和j连接，则代理i的收益为F[0,j]
            A[i,t,:] = F  #A:N*M*N   
            # F是三维矩阵A的第i页，第t行
            G[t,i] = F.dot(W[:,i]) #第t轮代理i的收益
        # update strategies
        for k in range(N):
            s=[i for i,x in enumerate(list(W[:,k])) if x>=1]     # 找出与代理k合作的代理的索引
            if len(s)!=0: # 如果有代理与代理k合作
                shuffle(s)
                P = 1/(1+math.e**((G[t,k]-G[t,s[0]])/K)) # 费米规则
                if random.random()<= P:
                    player[t+1,k] = player[t,s[0]]
                else:
                    player[t+1,k] = player[t,k]
            else: # 如果没有代理与代理k合作
                player[t+1,k] = player[t,k]
    # add noise for G
    Aa = G + Noise*random.random((M,N)) # m轮收益总矩阵：包含每一轮的收益M*N
    return [A,Aa]

In [3]:
def TrainSTRidge(R, Ut, lam, d_tol, maxit = 25, STR_iters = 10, l0_penalty = None, normalize = 2, split = 0.8, print_best_tol = False):
    """
    This function trains a predictor using STRidge.

    It runs over different values of tolerance and trains predictors on a training set, then evaluates them 
    using a loss function on a holdout set.

    Please note published article has typo.  Loss function used here for model selection evaluates fidelity using 2-norm,
    not squared 2-norm.
    """

    # Split data into 80% training and 20% test, then search for the best tolderance.
    np.random.seed(0) # for consistancy
    n,_ = R.shape
    train = np.random.choice(n, int(n*split), replace = False)
    test = [i for i in np.arange(n) if i not in train]
    TrainR = R[train,:]
    TestR = R[test,:]
    TrainY = Ut[train,:]
    TestY = Ut[test,:]
    D = TrainR.shape[1]       

    # Set up the initial tolerance and l0 penalty
    d_tol = float(d_tol)
    tol = d_tol
    if l0_penalty == None: l0_penalty = 0.001*np.linalg.cond(R)

    # Get the standard least squares estimator
    w = np.zeros((D,1))
    w_best = np.linalg.lstsq(TrainR, TrainY)[0]
    err_best = np.linalg.norm(TestY - TestR.dot(w_best), 2) + l0_penalty*np.count_nonzero(w_best)
    tol_best = 0

    # Now increase tolerance until test performance decreases
    for iter in range(maxit):

        # Get a set of coefficients and error
        w = STRidge(R,Ut,lam,STR_iters,tol,normalize = normalize)
        err = np.linalg.norm(TestY - TestR.dot(w), 2) + l0_penalty*np.count_nonzero(w)

        # Has the accuracy improved?
        if err <= err_best:
            err_best = err
            w_best = w
            tol_best = tol
            tol = tol + d_tol

        else:
            tol = max([0,tol - 2*d_tol])
            d_tol  = 2*d_tol / (maxit - iter)
            tol = tol + d_tol

    if print_best_tol: 
        print ("Optimal tolerance:", tol_best)
    return w_best

def STRidge(X0, y, lam, maxit, tol, normalize = 2, print_results = False):
    """
    Sequential Threshold Ridge Regression algorithm for finding (hopefully) sparse 
    approximation to X^{-1}y.  The idea is that this may do better with correlated observables.

    This assumes y is only one column
    """

    n,d = X0.shape
    X = np.zeros((n,d), dtype=np.complex64)
    # First normalize data
    if normalize != 0:
        Mreg = np.zeros((d,1))
        for i in range(0,d):
            Mreg[i] = 1.0/(np.linalg.norm(X0[:,i],normalize))
            X[:,i] = Mreg[i]*X0[:,i]
    else: X = X0
    
    # Get the standard ridge esitmate
    if lam != 0: w = np.linalg.lstsq(X.T.dot(X) + lam*np.eye(d),X.T.dot(y))[0]
    else: w = np.linalg.lstsq(X,y)[0]
    num_relevant = d
    biginds = np.where( abs(w) > tol)[0]
    
    # Threshold and continue
    for j in range(maxit):

        # Figure out which items to cut out
        smallinds = np.where( abs(w) < tol)[0]
        new_biginds = [i for i in range(d) if i not in smallinds]
            
        # If nothing changes then stop
        if num_relevant == len(new_biginds): break
        else: num_relevant = len(new_biginds)
            
        # Also make sure we didn't just lose all the coefficients
        if len(new_biginds) == 0:
            if j == 0: 
                #if print_results: print "Tolerance too high - all coefficients set below tolerance"
                return w
            else: break
        biginds = new_biginds
        
        # Otherwise get a new guess
        w[smallinds] = 0
        if lam != 0: w[biginds] = np.linalg.lstsq(X[:, biginds].T.dot(X[:, biginds]) + lam*np.eye(len(biginds)),X[:, biginds].T.dot(y))[0]
        else: w[biginds] = np.linalg.lstsq(X[:, biginds],y)[0]

    # Now that we have the sparsity pattern, use standard least squares to get w
    if biginds != []: w[biginds] = np.linalg.lstsq(X[:, biginds],y)[0]
    
    if normalize != 0: return np.multiply(Mreg,w)
    else: return w

In [4]:
M = 10 #m轮更新
b = 1.2 #叛逃者的收益
K = 0.1
P_PDG = array([[1,0],[b,0]]) #2*2的收益矩阵
Noise = 5
W = np.loadtxt("BA.txt") 
T = W.shape[0] #全部社区内个体的总数量
SV = 6
# the part for generate EG data
y = zeros((SV*M,T))
AA = zeros((T,SV*M,T))
for i in range(SV): #产生6组A和Y的值
    [A, Aa] = data_generate(M,P_PDG,W,T,K,Noise)
    for j in range(M):
        y[i*M+j,:] =Aa[j,:]
        for k in range(T):
            AA[k,i*M+j,:] = A[k,j,:]

In [5]:
print(Noise*random.random((M,T)))

[[1.33845485 3.36189134 2.10392411 1.40443759 0.49083518 0.52583495
  0.56784927 3.19346394 4.58449113 1.18602334 4.05936244 2.05324132
  1.46042735 0.76339307 3.81135628 1.26698443 1.97194928 2.35451691
  3.55674709 2.61008694 2.73560337 1.95760229 0.95104373 0.55914967
  3.09981463 4.73360205 0.07119996 0.57039477 0.43531548 3.67247532
  4.79370159 2.9727993  2.6800013  4.59873391 4.86367861 0.92342386
  2.90816554 0.2089273  3.42491307 1.13683283]
 [2.53483879 3.06248326 3.07430112 0.00879446 0.30816167 1.2056525
  1.24620025 0.20221674 0.66000523 0.66156344 1.87661507 4.47890572
  3.03618868 4.68262707 4.00295792 4.67839504 2.74793662 2.16653256
  0.22796125 2.94958728 3.11532857 3.85192962 4.13720909 3.36824033
  2.15563941 1.27401768 2.30145466 4.13351017 2.85726494 3.25200319
  1.1855454  3.97855322 2.69138172 1.73726176 4.2941575  0.77680128
  2.95886102 0.05763388 4.07088628 2.90945188]
 [2.5271408  3.06731424 4.75871306 3.15221433 3.91097349 3.95162677
  2.01886022 3.92691295

In [9]:
X=zeros((T,T))
for j in range(T):
    w = TrainSTRidge(AA[j], y[:,j].reshape(-1,1),10**0,1)
    for i in range(w.shape[0]):
        X[i,j]=w[i,0]
np.savetxt("X.txt", X,fmt='%f',delimiter=' ')

<ipython-input-3-13ac3e2d7f51>:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_best = np.linalg.lstsq(TrainR, TrainY)[0]
<ipython-input-3-13ac3e2d7f51>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X) + lam*np.eye(d),X.T.dot(y))[0]
<ipython-input-3-13ac3e2d7f51>:102: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advis

In [10]:
def TrainSTRidge_SPL(R, Ut, v, lam, d_tol, maxit = 25, STR_iters = 10, l0_penalty = None, normalize = 2, split = 0.8, print_best_tol = False):
    """
    This function trains a predictor using STRidge_SPL.

    It runs over different values of tolerance and trains predictors on a training set, then evaluates them 
    using a loss function on a holdout set.

    Please note published article has typo.  Loss function used here for model selection evaluates fidelity using 2-norm,
    not squared 2-norm.
    """

    # Split data into 80% training and 20% test, then search for the best tolderance.
    np.random.seed(0) # for consistancy
    n,_ = R.shape
    train = np.random.choice(n, int(n*split), replace = False)
    test = [i for i in np.arange(n) if i not in train]
    TrainR = R[train,:]
    TestR = R[test,:]
    TrainY = Ut[train,:]
    TestY = Ut[test,:]
    D = TrainR.shape[1]       

    # Set up the initial tolerance and l0 penalty
    d_tol = float(d_tol)
    tol = d_tol
    if l0_penalty == None: l0_penalty = 0.001*np.linalg.cond(R)

    # Get the standard least squares estimator
    w = np.zeros((D,1))
    w_best = np.linalg.lstsq(TrainR, TrainY)[0]
    err_best = np.linalg.norm(TestY - TestR.dot(w_best), 2) + l0_penalty*np.count_nonzero(w_best)
    tol_best = 0

    # Now increase tolerance until test performance decreases
    for iter in range(maxit):

        # Get a set of coefficients and error
        w = STRidge_SPL(R,Ut,v,lam,STR_iters,tol,normalize = normalize)
        err = np.linalg.norm(TestY - TestR.dot(w), 2) + l0_penalty*np.count_nonzero(w)

        # Has the accuracy improved?
        if err <= err_best:
            err_best = err
            w_best = w
            tol_best = tol
            tol = tol + d_tol

        else:
            tol = max([0,tol - 2*d_tol])
            d_tol  = 2*d_tol / (maxit - iter)
            tol = tol + d_tol

    if print_best_tol: 
        print ("Optimal tolerance:", tol_best)
    return w_best

def STRidge_SPL(X0, y, v, lam, maxit, tol, normalize = 2, print_results = False):
    """
    Sequential Threshold Ridge Regression algorithm for finding (hopefully) sparse 
    approximation to X^{-1}y.  The idea is that this may do better with correlated observables.

    This assumes y is only one column
    """

    n,d = X0.shape
    X = np.zeros((n,d), dtype=np.complex64)
    # First normalize data
    if normalize != 0:
        Mreg = np.zeros((d,1))
        for i in range(0,d):
            Mreg[i] = 1.0/(np.linalg.norm(X0[:,i],normalize))
            X[:,i] = Mreg[i]*X0[:,i]
    else: X = X0
    
    # Get the standard ridge esitmate
    if lam != 0: w = np.linalg.lstsq(X.T.dot(X*v) + lam*np.eye(d),X.T.dot(y*v))[0]
    else: w = np.linalg.lstsq(X*v,y*v)[0]
    num_relevant = d
    biginds = np.where( abs(w) > tol)[0]
    
    # Threshold and continue
    for j in range(maxit):

        # Figure out which items to cut out
        smallinds = np.where( abs(w) < tol)[0]
        new_biginds = [i for i in range(d) if i not in smallinds]
            
        # If nothing changes then stop
        if num_relevant == len(new_biginds): break
        else: num_relevant = len(new_biginds)
            
        # Also make sure we didn't just lose all the coefficients
        if len(new_biginds) == 0:
            if j == 0: 
                #if print_results: print "Tolerance too high - all coefficients set below tolerance"
                return w
            else: break
        biginds = new_biginds
        
        # Otherwise get a new guess
        w[smallinds] = 0
        if lam != 0: w[biginds] = np.linalg.lstsq(X[:, biginds].T.dot(X[:, biginds]*v) + lam*np.eye(len(biginds)),X[:, biginds].T.dot(y*v))[0]
        else: w[biginds] = np.linalg.lstsq(X[:, biginds]*v,y*v)[0]

    # Now that we have the sparsity pattern, use standard least squares to get w
    if biginds != []: w[biginds] = np.linalg.lstsq(X[:, biginds]*v,y*v)[0]
    
    if normalize != 0: return np.multiply(Mreg,w)
    else: return w

In [14]:
X=zeros((T,T))
v = ones((SV*M,1))
v1 = zeros((SV*M,1))
Los = zeros((SV*M,T))
losss = zeros((SV*M,1))
lambda_0=1
t=0
while (t<35):  
    if (v==v1).all():
        break
    else:
        for j in range(T):
            w = TrainSTRidge_SPL(AA[j], y[:,j].reshape(-1,1), v,10**0,1)
            Lo =  abs((np.dot(AA[j], w))-y[:,j].reshape(-1,1))
            for i in range(SV*M):
                Los[i,j]=Lo[i]
            for i in range(w.shape[0]):
                X[i,j]=w[i,0]
        for i in range(SV*M):
            losss[i]=np.mean(Los[i,:])
        t = t+1
        for i in range(SV*M):
            if losss[i]<lambda_0:
                v[i]=1-losss[i]/lambda_0
            else:
                v[i]=0
        lambda_0 = lambda_0*1.25
        #lambda_0 = lambda_0 + np.mean(losss)      
np.savetxt("X1.txt", X,fmt='%f',delimiter=' ')

<ipython-input-10-5e04d6e95ebc>:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_best = np.linalg.lstsq(TrainR, TrainY)[0]
<ipython-input-10-5e04d6e95ebc>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X*v) + lam*np.eye(d),X.T.dot(y*v))[0]
<ipython-input-10-5e04d6e95ebc>:102: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning w

In [15]:
v

array([[0.99964884],
       [0.9994012 ],
       [0.99950739],
       [0.99958157],
       [0.99947525],
       [0.99949577],
       [0.99946908],
       [0.9994305 ],
       [0.99942144],
       [0.99947414],
       [0.99948745],
       [0.99952782],
       [0.99948094],
       [0.99950939],
       [0.99945706],
       [0.99953031],
       [0.99953535],
       [0.99950843],
       [0.99945709],
       [0.99954756],
       [0.99959869],
       [0.9995633 ],
       [0.99956463],
       [0.99952797],
       [0.99943882],
       [0.99944487],
       [0.99958067],
       [0.99944768],
       [0.99943544],
       [0.99937078],
       [0.99957683],
       [0.99964048],
       [0.9995251 ],
       [0.99946462],
       [0.99952376],
       [0.99956155],
       [0.99947113],
       [0.99939002],
       [0.99957766],
       [0.99955122],
       [0.99965389],
       [0.99972118],
       [0.99951731],
       [0.99951068],
       [0.99954081],
       [0.99949439],
       [0.99949896],
       [0.999

In [20]:
X=zeros((T,T))
v = ones((SV*M,T))
v1 = zeros((SV*M,T))
Los = zeros((SV*M,T))
lambda_0=1
t=0
while (t<40):
    if (v==v1).all():
        break
    else:
        for j in range(T):
            w = TrainSTRidge_SPL(AA[j], y[:,j].reshape(-1,1), v[:,j].reshape(-1,1),10**0,1)
            Lo =  abs((np.dot(AA[j], w))-y[:,j].reshape(-1,1))
            for i in range(SV*M):
                Los[i,j]=Lo[i]
            for i in range(w.shape[0]):
                X[i,j]=w[i,0]
        t = t+1
        for j in range(T):
            for i in range(SV*M):
                if Los[i,j]<lambda_0:
                    v[i,j]=1-Los[i,j]/lambda_0
                else:
                    v[i,j]=0
        lambda_0 = lambda_0*1.25
        #lambda_0 = lambda_0 + np.mean(Los)
np.savetxt("X2.txt", X,fmt='%f',delimiter=' ')

<ipython-input-10-5e04d6e95ebc>:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_best = np.linalg.lstsq(TrainR, TrainY)[0]
<ipython-input-10-5e04d6e95ebc>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X*v) + lam*np.eye(d),X.T.dot(y*v))[0]
<ipython-input-10-5e04d6e95ebc>:102: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning w

In [18]:
v

array([[0.99993417, 0.9998871 , 0.99978017, ..., 0.99950231, 0.99999995,
        0.99978962],
       [0.99935015, 0.9990428 , 0.99972436, ..., 0.99934073, 0.99812392,
        0.99927207],
       [0.99977427, 0.99926651, 0.99961033, ..., 0.99979732, 0.99957917,
        0.99961069],
       ...,
       [0.99962752, 0.9996243 , 0.99973385, ..., 0.99854164, 0.99925716,
        0.999532  ],
       [0.99977251, 0.99931357, 0.99982283, ..., 0.99977256, 0.99902785,
        0.99888759],
       [0.99933667, 0.99938814, 0.99964254, ..., 0.99912681, 0.99927781,
        0.99970717]])